In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()

# Navigate to the web page
driver.get("https://devgan.in/crpc/chapter_38.php")

# Find all tags with class 'subClose' or 'subOpen'
section_tags = driver.find_elements(By.XPATH, '//h2[@class="subClose" or @class="subOpen"]')

# Iterate over each section tag, click on it, and extract the information
for section_tag in section_tags:
    # Scroll to the section tag using JavaScript
    driver.execute_script("arguments[0].scrollIntoView();", section_tag)

    # Click on the section tag using JavaScript to avoid ElementClickInterceptedException
    driver.execute_script("arguments[0].click();", section_tag)

    # Wait for the table to be present in the DOM
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="sectxt"]//table[@class="sch"]'))
    )

    # Extract rows from the table
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # Initialize lists to store the data
    data = []

    # Initialize variables to store header information
    headers = ['Section', 'Description', 'Punishment', 'Cognizance', 'Bail', 'Triable by']

    # Iterate over rows and extract data
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')
        if len(cells) >= 4:  # Check if there are enough cells
            section = cells[0].text.strip()
            description = cells[1].text.strip()
            punishment = cells[3].text.strip()

            # Append data to the list
            data.append([section, description, punishment])

    # Save the data to a CSV file for each section
    csv_file_path = f'output_section_{section_tag.text}.csv'
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(headers)
        csv_writer.writerows(data)

# Close the web driver
driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.217)
Stacktrace:
	GetHandleVerifier [0x00007FF638D92142+3514994]
	(No symbol) [0x00007FF6389B0CE2]
	(No symbol) [0x00007FF6388576AA]
	(No symbol) [0x00007FF638830AFD]
	(No symbol) [0x00007FF6388CCB1B]
	(No symbol) [0x00007FF6388E218F]
	(No symbol) [0x00007FF6388C5D93]
	(No symbol) [0x00007FF638894BDC]
	(No symbol) [0x00007FF638895C64]
	GetHandleVerifier [0x00007FF638DBE16B+3695259]
	GetHandleVerifier [0x00007FF638E16737+4057191]
	GetHandleVerifier [0x00007FF638E0E4E3+4023827]
	GetHandleVerifier [0x00007FF638AE04F9+689705]
	(No symbol) [0x00007FF6389BC048]
	(No symbol) [0x00007FF6389B8044]
	(No symbol) [0x00007FF6389B81C9]
	(No symbol) [0x00007FF6389A88C4]
	BaseThreadInitThunk [0x00007FFF2F7B257D+29]
	RtlUserThreadStart [0x00007FFF3096AA58+40]


updated code next button

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv

# Set up the Selenium WebDriver (make sure you have the appropriate webdriver installed, e.g., chromedriver)
driver = webdriver.Chrome()

# Navigate to the provided URL
url = "https://lawrato.com/indian-kanoon/ipc/section-121A"
driver.get(url)

# Initialize page count
page_count = 1

while True:
    # Extract the URL and write it to a CSV file
    current_url = driver.current_url

    with open('urls.csv', 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([page_count, current_url])

    # Locate and click the "Next" button if available
    try:
        next_button = driver.find_element(By.XPATH, '//a[@class="button button-primary pull-right" and contains(text(), "Next")]')
        next_button.click()
        page_count += 1
    except:
        # If no "Next" button is found, break out of the loop
        break

# Close the WebDriver
driver.quit()


In [ ]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

def extract_information(url):
    # Set up the Selenium WebDriver
    driver = webdriver.Chrome()

    # Initialize variables
    description_text = None
    offense_text = None
    punishment_text = None
    cognizable_text = None
    bailable_text = None
    court_text = None

    try:
        # Load the webpage
        driver.get(url)

        # Find the element containing the description information
        description_element = driver.find_element("id", "description-container")
        description_text = description_element.text

        # Find the table containing the legal details
        table_element = driver.find_element(By.XPATH, "//tbody[@style='border:1px !important;']")

        # Extract the HTML content from the table
        table_html = table_element.get_attribute("outerHTML")

        # Use BeautifulSoup for parsing HTML
        soup = BeautifulSoup(table_html, "html.parser")

        # Extract rows from the table
        rows = soup.find_all("tr")

        # Extract and structure legal details
        legal_details = []
        for row in rows:
            columns = row.find_all("td")
            details = [col.text.strip() for col in columns]
            legal_details.append(details)

        # Extract individual details
        offense_text = legal_details[0][0] if legal_details else None
        punishment_text = legal_details[0][1] if legal_details else None
        cognizable_text = legal_details[0][2] if legal_details else None
        bailable_text = legal_details[0][3] if legal_details else None
        court_text = legal_details[0][4] if legal_details else None

    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}")

    finally:
        # Close the browser
        driver.quit()

    return description_text, offense_text, punishment_text, cognizable_text, bailable_text, court_text

# Read URLs from the CSV file
with open("urls.csv", mode="r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    urls = [row["URL"] for row in reader]

# Process each URL and save information to a separate CSV file
for index, url in enumerate(urls, start=1):
    description_text, offense_text, punishment_text, cognizable_text, bailable_text, court_text = extract_information(url)
    csv_data = [["URL", "Description", "Offense", "Punishment", "Cognizable", "Bailable", "Court"]]
    
    csv_data.append([url, description_text, offense_text, punishment_text, cognizable_text, bailable_text, court_text])

    # Create a separate CSV file for each page
    output_filename = f"output_page_{index}.csv"
    with open(output_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)
